# Load packages

In [1]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
    
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt

from scipy import optimize
from scipy.optimize import minimize
from scipy.stats import gumbel_r
from matplotlib.pyplot import cm
import math
import matplotlib
import matplotlib.colors as mcolors
from scipy.optimize import fsolve

import sympy
from sympy import *
from sympy import symbols, Eq, solve
from shapely.geometry import LineString


%matplotlib inline
plt.style.use('ggplot')
import warnings
from scipy.stats import pearsonr

# Catchment selection 

In [2]:
"""
LamaH: Catchment selection
The data set contains a selection of catchments with no human impact.
Snow fraction is checked by frac_snow < 0.1

"""

frac_snow = pd.read_csv('1_Data/LamaH/C_basins_intermediate_lowimp/1_attributes/Catchment_attributes.csv', delimiter=';', parse_dates=[0,1,2], skipinitialspace=True)
# display(frac_snow)

catchment_list_Lamah = []
for i in range(len(frac_snow)):
    if (frac_snow['frac_snow'][i]<0.1):
        catchment_list_Lamah.append(int(frac_snow['ID'][i]))
        
# print(catchment_list_Lamah)

# Catchment descriptors 

In [5]:
#FUNCTIONS to compute the catchment descriptors.

#HAI
def HAI_calculate(temp, precipitation):

    temp_adjusted = np.zeros(12)
    
    for i in range(1,13):
        if temp[i] < 0:
            temp_adjusted[i-1] = 0
        elif temp[i] > 30:
            temp_adjusted[i-1] = 30
        else:
            temp_adjusted[i-1] = temp[i]            
        
    T = np.sum(temp_adjusted)
    HAI = (58.93 * (T/12))/precipitation
    
    return HAI


#Function for the Seasonality Index
def Seasonality_index(Annual_mean, Monthly_mean):
    SI = (1/Annual_mean) * np.sum(np.abs(Monthly_mean - (Annual_mean/12)))
    
    return SI

#Function for Seasonality Timing Index
def ST_calc(dP,dT):
    days = 366
    ST = dP[0] * np.sign(dT[0]) * np.cos((np.pi * (dP[1] - dT[1]))/days)
    return ST

#Functions to compute Seasonal variability indexes
def T_daily(dT):
    t = np.linspace(1,366,366)
    days = 366
    T = T_mean + dT[0] * np.sin((2*np.pi * (t-dT[1]))/days)
    return T

def Cal_T_daily(dT):
    days = 366
    T_calc = T_daily(dT)
    
    return (np.sum(np.abs(T_calc - T_obs)))/days

def P_daily(dP):
    t = np.linspace(1,366,366)
    days = 366
    P = P_mean * (1 + dP[0] * np.sin((2*np.pi * (t-dP[1]))/days))
    return P

def Cal_P_daily(dP):
    days = 366
    P_calc = P_daily(dP)
    
    return (np.sum(np.abs(P_calc - P_obs)))/days

def E_daily(dE):
#    t = np.linspace(1,366,366)
    t = np.linspace(1,366,366)

    days = 366
    E = E_mean * (1 + dE[0] * np.sin((2*np.pi * (t-dE[1]))/days))
    return E

def Cal_E_daily(dE):
    days = 366
    E_calc = E_daily(dE)
    
    return (np.sum(np.abs(E_calc - E_obs)))/days


#Function for the interstorm duration or low_prec_freq
def Interstorm_duration(P):
    interstorm = []
    count = 0

    for j in range(len(P)):
        if P[j] < 1:
            count += 1
        elif P[j] >= 1 and count > 0:
            interstorm.append(count)
            count = 0 
    interstorm_duration = np.mean(interstorm)
    return 

#Function to compute the root-zone storage capacity
def rootzone(df_out, RP):
    firstyear = df_out['hydroyear'].iloc[0]
    lastyear = df_out['hydroyear'].iloc[-1]
    df_out = df_out.drop(df_out[df_out['hydroyear']==firstyear].index)
    df_out = df_out.drop(df_out[df_out['hydroyear']==lastyear].index)
    years = df_out.groupby(['hydroyear'])
    SD = years['SD_sum'].max() - years['SD_sum'].min()
    var = gumbel_r.fit(SD)
    p = 1-1/RP
    Sr = gumbel_r.ppf(p, var[0],var[1])
    return(Sr, df_out)

## Load et0_mean

# LamaH: Load data

In [4]:
catch_att = pd.read_csv('1_Data/LamaH/C_basins_intermediate_lowimp/1_attributes/Catchment_attributes.csv', delimiter=';', parse_dates=[0,1,2], skipinitialspace=True)
catch_att.index = catch_att['ID']

In [10]:
catchment_list = catchment_list_Lamah

for i in range(len(catchment_list)): 
    # Import data and select right dates
    data = pd.read_csv('1_Data/LamaH/C_basins_intermediate_lowimp/2_timeseries/daily/'+str('ID_')+str(catchment_list[i])+str('.csv'),delimiter=';', skipinitialspace=True)
    data['date'] = pd.to_datetime(data['YYYY'].astype('str')+
                                 data['MM'].astype('str')+
                                 data['DD'].astype('str'),
                           format='%Y%m%d')
    data.loc[:,'dt'] = pd.to_datetime(data['date'])
    data.index = data['dt']
    data = data["1989-01-01":"2009-12-31"]
    
    
    outputdata = pd.read_csv('1_Data/LamaH/F_hydrol_model/2_timeseries/'+str('ID_')+str(catchment_list[i])+str('.csv'),delimiter=';', skipinitialspace=True)
    outputdata['date'] = pd.to_datetime(outputdata['YYYY'].astype('str')+
                                 outputdata['MM'].astype('str')+
                                 outputdata['DD'].astype('str'),
                                        format='%Y%m%d')
    outputdata.loc[:,'dt'] = pd.to_datetime(outputdata['date'])
    outputdata.index = outputdata['dt']
    outputdata = outputdata["1989-01-01":"2009-12-31"]   
    
    data["qobs"] = outputdata["Qobs"]    
    
    # Compute mean values
    Ep = catch_att.loc[f"{catchment_list[i]}","et0_mean"]  # comes from catchment attributes 
    P = data['prec'].mean() 
    T = data['2m_temp_mean'].mean() 
    Q = data['qobs'].mean()
    Ea = data['total_et'].mean()
    
    # Compute Ea, AI, EI
    Ea = P - Q
    EI = Ea / P
    AI = Ep / P

2.28 3.0608865710560598 8.473142112125142 1.1319556714471861 1.9289308996088737


# Plot Budyko

In [7]:
catchment_list = catchment_list_Lamah
plt.figure(figsize=(10,5)).suptitle("LamaH (n=59)",fontsize=20)
n_total = 0

for i in range(len(catchment_list)): 
    # Import data and select right dates
    data = pd.read_csv('1_Data/LamaH/C_basins_intermediate_lowimp/2_timeseries/daily/'+str('ID_')+str(catchment_list[i])+str('.csv'),delimiter=';', skipinitialspace=True)
    data['date'] = pd.to_datetime(data['YYYY'].astype('str')+
                                 data['MM'].astype('str')+
                                 data['DD'].astype('str'),
                           format='%Y%m%d')
    data.loc[:,'dt'] = pd.to_datetime(data['date'])
    data.index = data['dt']
    data = data["1989-01-01":"2009-12-31"]
    
    
    outputdata = pd.read_csv('1_Data/LamaH/F_hydrol_model/2_timeseries/'+str('ID_')+str(catchment_list[i])+str('.csv'),delimiter=';', skipinitialspace=True)
    outputdata['date'] = pd.to_datetime(outputdata['YYYY'].astype('str')+
                                 outputdata['MM'].astype('str')+
                                 outputdata['DD'].astype('str'),
                                        format='%Y%m%d')
    outputdata.loc[:,'dt'] = pd.to_datetime(outputdata['date'])
    outputdata.index = outputdata['dt']
    outputdata = outputdata["1989-01-01":"2009-12-31"]   
    
    data["qobs"] = outputdata["Qobs"]    
    
    # Compute mean values
    Ep = catch_att.loc[f"{catchment_list[i]}","et0_mean"]  # comes from catchment attributes 
    P = data['prec'].mean() 
    T = data['2m_temp_mean'].mean() 
    Q = data['qobs'].mean()
    Ea_calc = data['total_et'].mean()
    
    # display(data)
    
    # Compute Ea, AI, EI
    Ea = P - Q
    EI = Ea / P
    AI = Ep / P
    
    print(f'Catchment nr {i} has a given Ea of {Ea_calc} and Ea from P-Q of {Ea}, EI of {EI} and AI of {AI}')
        
        
    # Plot every catchment
    if Ea > 0 and AI > EI: 
        budyko_curve_x = np.arange(1, 3, 0.05)
        energy_limit_x = np.arange(0, 1.0001, 0.05)
        x = np.arange(0, 1.0001, 0.05)
        water_limit_y = 1 + budyko_curve_x*0
        energy_limit_y = energy_limit_x
        y = 1 + x*0
        plt.ylabel("Actual ET/P")
        plt.xlabel("Potential ET/P")
        plt.minorticks_on()
        plt.plot(energy_limit_x, energy_limit_y, c='k')
        plt.plot(budyko_curve_x, water_limit_y,c='k')
        
        plt.plot(AI, EI, marker='.', c = 'b')
        n_total += 1

        
plt.savefig(f'2_Output/Output3/Budyko.png')   
plt.close()

print(n_total)

Catchment nr 0 has a given Ea of 1.704422425032596 and Ea from P-Q of 1.2975293350716823, EI of 0.29613018236373434 and AI of 0.5386138189621771
Catchment nr 1 has a given Ea of 1.6221408083441957 and Ea from P-Q of 3.057956975228156, EI of 0.8042332559772655 and AI of 0.6022629393111485
Catchment nr 2 has a given Ea of 1.6791655801825316 and Ea from P-Q of 2.7714576271186377, EI of 0.772998219605287 and AI of 0.6331346874418181
Catchment nr 3 has a given Ea of 1.6111877444589378 and Ea from P-Q of 2.734398956975211, EI of 0.7250936236357488 and AI of 0.6045984832141283
Catchment nr 4 has a given Ea of 1.7969100391134345 and Ea from P-Q of -1.0682086049543655, EI of -0.30419519794578387 and AI of 0.6407355194515152
Catchment nr 5 has a given Ea of 1.752791395045628 and Ea from P-Q of 0.12822294654497401, EI of 0.0384935584436915 and AI of 0.6724659917530866
Catchment nr 6 has a given Ea of 1.824807040417211 and Ea from P-Q of 2.695640156453706, EI of 0.7901501989939853 and AI of 0.6565